#### OpenAI API Setup and Authentication

In [ ]:
import openai
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

class AIDataAnalyst:
    """AI-powered data analysis using OpenAI API"""
    
    def __init__(self, api_key=None):
        # Initialize the OpenAI client (v1.0+ syntax)
        if api_key:
            self.client = OpenAI(api_key=api_key)
        else:
            self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        
        if not self.client.api_key:
            raise ValueError("OpenAI API key not found. Set OPENAI_API_KEY environment variable.")
    
    def test_connection(self):
        """Test OpenAI API connection"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "user", "content": "Say 'API connection successful' if you can read this."}
                ],
                max_tokens=10
            )
            print("✅ OpenAI API connection successful!")
            print(f"Response: {response.choices[0].message.content}")
            return True
        except Exception as e:
            print(f"❌ API connection failed: {e}")
            return False

# Test your setup
try:
    ai_analyst = AIDataAnalyst()
    ai_analyst.test_connection()
except Exception as e:
    print(f"Failed to initialize: {e}")
    print("Make sure your OPENAI_API_KEY is set in your .env file")

#### Basic Data-to-Insights Pattern

In [4]:
def analyze_dataframe_with_ai(df, question="What insights can you provide about this data?"):
    """
    Send DataFrame to AI for analysis
    This is the core pattern you'll use everywhere
    """
    
    # Convert DataFrame to a readable format for AI
    data_summary = {
        'shape': df.shape,
        'columns': list(df.columns),
        'sample_data': df.head(3).to_dict('records'),
        'statistics': df.describe().to_dict() if len(df.select_dtypes(include=['number']).columns) > 0 else "No numeric columns"
    }
    
    prompt = f"""
    I have a dataset with the following characteristics:
    
    Shape: {data_summary['shape'][0]} rows, {data_summary['shape'][1]} columns
    Columns: {data_summary['columns']}
    
    Sample data (first 3 rows):
    {json.dumps(data_summary['sample_data'], indent=2, default=str)}
    
    Statistical summary:
    {json.dumps(data_summary['statistics'], indent=2, default=str)}
    
    Question: {question}
    
    Please provide:
    1. Key insights about the data
    2. Notable patterns or trends
    3. Potential business implications
    4. Recommendations for further analysis
    
    Keep your response business-focused and actionable.
    """
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a senior data analyst who provides clear, actionable insights from business data."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.3  # Lower temperature for more consistent analysis
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Error analyzing data: {e}"

# Test with sample data
import sqlite3

# Load some data from your database
def get_sample_sales_data():
    """Get sample data for AI analysis"""
    conn = sqlite3.connect('business_data.db')
    query = """
    SELECT 
        c.region,
        COUNT(o.order_id) as order_count,
        SUM(o.total_amount) as total_revenue,
        AVG(o.total_amount) as avg_order_value
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    WHERE o.status = 'Completed'
    GROUP BY c.region
    ORDER BY total_revenue DESC
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Analyze your sales data with AI
sales_data = get_sample_sales_data()
ai_insights = analyze_dataframe_with_ai(sales_data, "What do these regional sales patterns tell us about our business performance?")
print("AI Analysis of Sales Data:")
print(ai_insights)

AI Analysis of Sales Data:
Error analyzing data: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

